In [2]:
from ultralytics import YOLO
import torch
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import shutil
from glob import glob

In [2]:
base_path = '../../dataset/coronal'
train_image = os.path.join(base_path, 'images/train')
train_label = os.path.join(base_path, 'labels/train')
test_image = os.path.join(base_path, 'images/test')
test_label = os.path.join(base_path, 'labels/test')
val_image = os.path.join(base_path, 'images/val')
val_label = os.path.join(base_path, 'labels/val')

In [3]:
# seed 및 GPU 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seed(42)

# Read and sort the file list to ensure consistent order
file_list = sorted(os.listdir(train_image))
random_file = random.choice(file_list).split('.')[0]
print("Image :", random_file)

# Read labels from the file
with open(os.path.join(train_label, f'{random_file}.txt'), 'r') as f:
    labels = f.readlines()
    labels = labels[0].split(' ')

# Read and process the image
img = cv2.imread(os.path.join(train_image, f'{random_file}.jpg'), cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Image : 78 (5)


In [4]:
# 파일 개수 세는 함수
def count_files(directory):
    return len(os.listdir(directory))

num_train_images = count_files(train_image)
num_test_images = count_files(test_image)
num_train_labels = count_files(train_label)
num_test_labels = count_files(test_label)
print(f'Train images: {num_train_images}, Train labels: {num_train_labels}')
print(f'Test images: {num_test_images}, Test labels: {num_test_labels}\n')

Train images: 255, Train labels: 255
Test images: 78, Test labels: 78



In [5]:
#실행XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

# 라벨 없는 데이터 삭제
# if num_train_images != num_train_labels:
#     train_images = os.listdir(train_image)
#     train_labels = os.listdir(train_label)
#     labelled_images = set([os.path.splitext(file)[0] + '.jpg' for file in train_labels])

#     라벨 없는 이미지 제거
#     unlabelled_images = set(train_images) - labelled_images
#     for image in unlabelled_images:
#         os.remove(os.path.join(train_image, image))
        
# num_train_images = count_files(train_image)
# num_test_images = count_files(test_image)
# num_train_labels = count_files(train_label)
# num_test_labels = count_files(test_label)
# print(f'Train images: {num_train_images}, Train labels: {num_train_labels}')
# print(f'Test images: {num_test_images}, Test labels: {num_test_labels}\n')

In [6]:
#실행XXXXXXXXXXXXXXXXXXXXXXXXXXXXXxxxxx

# val_image = os.path.join(base_path, 'images/val')
# val_label = os.path.join(base_path, 'labels/val')

# # Get list of train images
# train_images = os.listdir(train_image)

# # Calculate the number of validation images
# num_train_images = len(train_images)
# val_split = int(num_train_images * 0.2)

# # Sample validation images from train images
# random.seed(42)
# val_images = random.sample(train_images, val_split)

# # Remove existing val directories if they exist
# if os.path.exists(val_image):
#     shutil.rmtree(val_image)
# if os.path.exists(val_label):
#     shutil.rmtree(val_label)

# # Create new val directories
# os.makedirs(val_image, exist_ok=True)
# os.makedirs(val_label, exist_ok=True)

# # Move selected images and corresponding labels to val directories
# for image in val_images:
#     shutil.move(os.path.join(train_image, image), val_image)
#     label = os.path.splitext(image)[0] + '.txt'
#     shutil.move(os.path.join(train_label, label), val_label)

# # Debugging: Print the count of files in each directory
# print("Train images:", len(os.listdir(train_image)))
# print("Validation images:", len(os.listdir(val_image)))
# print("Train labels:", len(os.listdir(train_label)))
# print("Validation labels:", len(os.listdir(val_label)))

In [8]:
#v8n
experiment_name = 'train'
work_name = 'base' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

model_n = YOLO('yolov8n.pt') #이름수정주의 2개
results_n = model_n.train( #이름수정주의 2개
    data='coronal_t1wce_2_class.yaml', 
    epochs=200, 
    device=0,  
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),  
    exist_ok=True,
)

'''
def delete_unwanted_files(train_dir, extension=".jpg"):
    jpg_files = glob(os.path.join(train_dir, f'*{extension}'))
    for file_path in jpg_files:
        if os.path.exists(file_path):
            os.remove(file_path)

# Call the function to delete unwanted .jpg files
delete_unwanted_files(train_dir, extension=".jpg")
'''

100%|██████████| 6.25M/6.25M [00:00<00:00, 11.7MB/s]


Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coronal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train_base, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /home/under1/Detect/dataset/coronal/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/coronal/labels/val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_base/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_base
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      3.38G      1.705      4.702      1.462         27        640: 100%|██████████| 16/16 [00:49<00:00,  3.09s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.70it/s]

...

                   all         63         66       0.92      0.815      0.864      0.668



200 epochs completed in 0.136 hours.
Optimizer stripped from runs/detect/train/train_base/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_base/weights/best.pt, 6.3MB

Validating runs/detect/train/train_base/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.38it/s]


                   all         63         66      0.931      0.849      0.885      0.689
              negative         36         39       0.95      0.846      0.924      0.729
              positive         27         27      0.913      0.852      0.846       0.65
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train/train_base


'\ndef delete_unwanted_files(train_dir, extension=".jpg"):\n    jpg_files = glob(os.path.join(train_dir, f\'*{extension}\'))\n    for file_path in jpg_files:\n        if os.path.exists(file_path):\n            os.remove(file_path)\n\n# Call the function to delete unwanted .jpg files\ndelete_unwanted_files(train_dir, extension=".jpg")\n'

In [9]:
results_n.results_dict

{'metrics/precision(B)': 0.9312058266392276,
 'metrics/recall(B)': 0.849002849002849,
 'metrics/mAP50(B)': 0.8850947244343502,
 'metrics/mAP50-95(B)': 0.689334214145704,
 'fitness': 0.7089102651745686}

In [3]:
#val
experiment_name = 'val'
work_name = 'base' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/coronal/runs/detect/train/train_base/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/coronal/labels/val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.07it/s]


                   all         63         66      0.932      0.849      0.884      0.689
              negative         36         39       0.95      0.846      0.925      0.728
              positive         27         27      0.913      0.852      0.844      0.651
Speed: 3.4ms preprocess, 4.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/val/val_base


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fba45bd1db0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [4]:
# test
experiment_name = 'test'
work_name = 'base' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/coronal/labels/test.cache... 78 images, 0 backgrounds, 0 corrupt: 100%|██████████| 78/78 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.61it/s]


                   all         78         83      0.512      0.805      0.562      0.424
              negative         32         34      0.477      0.794      0.556      0.441
              positive         46         49      0.546      0.816      0.568      0.406
Speed: 2.6ms preprocess, 2.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/test/test_base


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fb8a2dce8f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804